In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [11]:


# 데이터 불러오기 및 가공
data = pd.read_csv('test_data.csv')
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['Timestamp'] = data['Timestamp'].astype('int64') // 10**9
data = data[['Timestamp', 'uiSpeed', 'vehicleSpeed', 'acc_value','Physical_value']]

# 데이터 정규화
scaler = MinMaxScaler()
data_normalized = scaler.fit_transform(data)
print(f"data_normalized.shape: {data_normalized.shape}")

# 시퀀스 생성
sequence_length = 150  # 시퀀스 길이
X = []
y = []

# acc_value를 타겟으로 설정 (data_normalized의 마지막 열)
for i in range(len(data_normalized) - sequence_length):
    X.append(data_normalized[i:i+sequence_length, :-1])
    y.append(data_normalized[i+sequence_length, -1])  # acc_value를 타겟으로 사용

X = np.array(X)
y = np.array(y)

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tensor 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# 데이터셋 및 데이터로더 생성
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 모델 불러오기



data_normalized.shape: (3192, 5)


In [12]:
# LSTM 모델 정의
class LSTMBinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.7):
        super(LSTMBinaryClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        # LSTM 레이어 정의
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)  # 출력 레이

    def forward(self, x):
        # LSTM의 초기 은닉 상태와 셀 상태 초기화
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # LSTM 레이어 통과 (h0, c0 초기 은닉 상태와 셀 상태)
        out, _ = self.lstm(x, (h0, c0))
        
        # 최종 LSTM 은닉 상태 출력만 사용
        out = self.fc(out[:, -1, :])  # 마지막 타임스텝의 출력 사용
        return out

# 하이퍼파라미터 설정
input_size = 4  # 입력 피처의 개수 (Timestamp, DI_uiSpeed, DI_vehicleSpeed, Physical_value)
hidden_size = 128  # LSTM의 은닉 상태 크기
num_layers = 5  # LSTM 레이어 수
output_size = 1  # 이진분류 출력 크기
dropout = 0.7  # 드롭아웃 확률

# LSTM 모델 생성
model = LSTMBinaryClassifier(input_size, hidden_size, num_layers, output_size, dropout)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [16]:
model = LSTMBinaryClassifier(input_size, hidden_size, num_layers, output_size)
model.load_state_dict(torch.load('model_weights2.pth'))
model.eval()  # 모델을 평가 모드로 전환 (학습 x)

# 예측값과 실제값을 저장할 리스트를 만듭니다.
all_preds = []
all_labels = []

start_time = time.time()

# 데이터에 대해 예측 진행
with torch.no_grad():  # 학습이 아닌 평가이므로 gradient 계산 안 함
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)  # 가장 높은 확률의 클래스를 예측
        all_preds.extend(preds.numpy())
        all_labels.extend(labels.numpy())
    
    

end_time = time.time()
elapsed_time = end_time - start_time

all_labels = [int(label) for label in all_labels]
# Confusion Matrix와 Accuracy 계산
cm = confusion_matrix(all_labels, all_preds)
accuracy = accuracy_score(all_preds, all_labels)
precision = precision_score(all_preds, all_labels)
recall = recall_score(all_preds, all_labels)
f1 = f1_score(all_preds, all_labels)


# 결과 출력
print("Confusion Matrix:\n", cm)
print(f"Prediction time: {elapsed_time:.2f} seconds")

C:\Users\mmn10\AppData\Local\Temp\ipykernel_18464\1808465388.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_weights2.pth'))


Confusion Matrix:
 [[607   0]
 [  2   0]]
Prediction time: 0.48 seconds


C:\Users\mmn10\AppData\Local\Temp\ipykernel_18464\1808465388.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  all_labels = [int(label) for label in all_labels]
c:\Users\mmn10\anaconda3\envs\torch\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
